<a href="https://colab.research.google.com/github/adarsh8511/Call-Quality-Analyzer/blob/main/Call_Quality_Analyzer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🔧 Install required libraries
!pip install faster-whisper pydub noisereduce torch soundfile nltk -q
!pip install vaderSentiment -q

# 🔧 Download NLTK resources
import nltk
nltk.download("punkt")

# Step 1: Preprocess Audio (Noise Reduction + Resample to 16kHz mono)
import noisereduce as nr
import soundfile as sf
import numpy as np
from scipy.signal import resample_poly

def preprocess_audio(input_path, output_path="cleaned.wav"):
    audio, sr = sf.read(input_path)
    if audio.ndim > 1:  # stereo to mono
        audio = np.mean(audio, axis=1)
    # Downsample to 16kHz for faster processing
    if sr != 16000:
        audio = resample_poly(audio, 16000, sr)
        sr = 16000
    reduced = nr.reduce_noise(y=audio, sr=sr)
    sf.write(output_path, reduced, sr)
    return output_path

# Step 2: Transcribe with Whisper
from faster_whisper import WhisperModel

asr_model = WhisperModel("tiny", device="cpu")

def transcribe(audio_path):
    segments, info = asr_model.transcribe(audio_path, beam_size=1)
    transcript = []
    for seg in segments:
        transcript.append({
            "start": seg.start, "end": seg.end, "text": seg.text.strip()
        })
    return transcript

# Step 3: Simple Speaker Turn Detection
# Heuristic: alternate speakers when silence >1.5s
def assign_speakers(transcript, pause_threshold=1.5):
    merged = []
    current_speaker = "SPEAKER_0"
    prev_end = None
    for t in transcript:
        if prev_end is not None and (t["start"] - prev_end) > pause_threshold:
            current_speaker = "SPEAKER_1" if current_speaker == "SPEAKER_0" else "SPEAKER_0"
        merged.append({**t, "speaker": current_speaker})
        prev_end = t["end"]
    return merged

# Step 4: Analyze Call
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

def analyze_call(merged):
    analyzer = SentimentIntensityAnalyzer()
    results = {}

    # --- Talk-time ratio ---
    durations = {}
    for m in merged:
        dur = m["end"] - m["start"]
        durations[m["speaker"]] = durations.get(m["speaker"], 0) + dur
    total = sum(durations.values()) if durations else 1
    talk_ratio = {spk: round(100*dur/total,1) for spk,dur in durations.items()}

    # --- Number of questions ---
    num_questions = sum(1 for m in merged if "?" in m["text"])

    # --- Longest monologue ---
    longest = max([m["end"]-m["start"] for m in merged]) if merged else 0

    # --- Sentiment ---
    all_text = " ".join([m["text"] for m in merged])
    score = analyzer.polarity_scores(all_text)
    if score["compound"] > 0.2:
        sentiment = "Positive"
    elif score["compound"] < -0.2:
        sentiment = "Negative"
    else:
        sentiment = "Neutral"

    # --- Actionable insight ---
    if num_questions < 3:
        insight = "Ask more discovery questions to engage the customer."
    elif talk_ratio and max(talk_ratio.values()) > 70:
        insight = "Reduce monologue; let the customer speak more."
    else:
        insight = "Good balance of engagement and listening."

    results.update({
        "Talk Ratio": talk_ratio,
        "Questions Asked": num_questions,
        "Longest Monologue (s)": round(longest,2),
        "Sentiment": sentiment,
        "Actionable Insight": insight
    })
    return results

# Step 5: Identify Sales Rep vs Customer (Heuristic)
def identify_roles(analysis):
    if not analysis["Talk Ratio"]:
        return {}
    rep = max(analysis["Talk Ratio"], key=analysis["Talk Ratio"].get)
    customer = [s for s in analysis["Talk Ratio"].keys() if s != rep][0] if len(analysis["Talk Ratio"]) > 1 else "Unknown"
    return {"Sales Rep": rep, "Customer": customer}

# Step 6: Run Everything
from google.colab import files
uploaded = files.upload()
file_path = list(uploaded.keys())[0]

# --- Run pipeline ---
cleaned = preprocess_audio(file_path)
transcript = transcribe(cleaned)
merged = assign_speakers(transcript)
analysis = analyze_call(merged)
roles = identify_roles(analysis)

print("===== Call Quality Report =====")
for k,v in analysis.items():
    print(f"{k}: {v}")
print("Roles:", roles)



[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Saving Sales-Call-example-1.mp3 to Sales-Call-example-1 (4).mp3
===== Call Quality Report =====
Talk Ratio: {'SPEAKER_0': 100.0}
Questions Asked: 5
Longest Monologue (s): 11.0
Sentiment: Positive
Actionable Insight: Reduce monologue; let the customer speak more.
Roles: {'Sales Rep': 'SPEAKER_0', 'Customer': 'Unknown'}


In [1]:
!pip install fastapi uvicorn pyngrok python-multipart -q
!pip install faster-whisper pydub noisereduce torch soundfile nltk vaderSentiment -q

import nltk
nltk.download("punkt")


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 17.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.8/38.8 MB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.5/16.5 MB 63.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 5.2 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [4]:
!ngrok config add-authtoken 31yczibPNHDRyoqFVPjbd0CvZiN_4LWcojQXNdXsrnymipMRV


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [5]:
public_url = ngrok.connect(8000)
print(public_url)

NgrokTunnel: "https://d4b2c983fffc.ngrok-free.app" -> "http://localhost:8000"
